# COLLECTING DATA FROM RAIDER.IO (WOW RANKINGS WEBSITE)

## API Raider.io

API obtained from 'https://raider.io/api#/mythic_plus/getApiV1MythicplusRuns', which generates the API's URL according to your needs.

In [1]:
import requests
import pandas as pd
import json

## The API is public, so the connection process was pretty straight forward. 

In [2]:
#checking the structure of the website. This way we could see that each key "rankings" was a page from the website.
response = requests.get('https://raider.io/api/v1/mythic-plus/runs?season=season-7.3.0&region=world&dungeon=all&page=0')
results = response.json()
results

{'rankings': [{'rank': 1,
   'score': 381.7,
   'run': {'season': 'season-7.3.0',
    'status': 'finished',
    'dungeon': {'id': 999998,
     'name': 'Return to Karazhan: Lower',
     'short_name': 'LOWR',
     'slug': 'return-to-karazhan-lower',
     'expansion_id': 6,
     'patch': '7.2',
     'keystone_timer_ms': 2520999,
     'num_bosses': 4,
     'group_finder_activity_id': 471},
    'keystone_run_id': 3836767,
    'mythic_level': 24,
    'clear_time_ms': 2196695,
    'keystone_time_ms': 2340999,
    'completed_at': '2017-11-21T05:17:38.000Z',
    'num_chests': 1,
    'time_remaining_ms': 144304,
    'logged_run_id': None,
    'weekly_modifiers': [{'id': 8,
      'icon': 'spell_shadow_bloodboil',
      'name': 'Sanguine',
      'description': 'When slain, non-boss enemies leave behind a lingering pool of ichor that heals their allies and damages players.'},
     {'id': 12,
      'icon': 'ability_backstab',
      'name': 'Grievous',
      'description': 'Injured players suffer inc

## Extracting the characters parameters (role, name, class)

In [3]:
#testing a loop that extracts a dataframe in which each line is a player, starting with the first run of the first page

results = []

results.append(requests.get(f'https://raider.io/api/v1/mythic-plus/runs?season=season-7.3.0&region=world&dungeon=all&page=0').json())

pd.json_normalize(results[0]['rankings'][0]['run']['roster'])

,oldCharacter,isTransfer,role,character.id,character.persona_id,character.name,character.class.id,character.class.name,character.class.slug,character.race.id,...,character.realm.altName,character.realm.slug,character.realm.altSlug,character.realm.locale,character.realm.isConnected,character.region.name,character.region.slug,character.region.short_name,character.stream,character.recruitmentProfiles
0,None,True,dps,1744,508,Imfiredup-1744,8,Mage,mage,8,...,None,illidan,illidan,en_US,False,United States & Oceania,us,US,None,[]
1,None,True,tank,1282089,2678248,Zqs-1282089,10,Monk,monk,10,...,None,tichondrius,tichondrius,en_US,False,United States & Oceania,us,US,None,[]
2,None,False,dps,9311198,2110136,Wafflesauce,11,Druid,druid,6,...,None,tichondrius,tichondrius,en_US,False,United States & Oceania,us,US,None,[]
3,None,True,dps,9545803,355504,Níghtfall-9545803,9,Warlock,warlock,8,...,None,tichondrius,tichondrius,en_US,False,United States & Oceania,us,US,None,[]
4,None,True,healer,10279382,173963,Maset-10279382,2,Paladin,paladin,10,...,None,thrall,thrall,en_US,False,United States & Oceania,us,US,None,[]


In [4]:
#loop to extract all the names of the players from the first 100 pages of a specific season, an error was found on page 13,
#since there was a run with only 3 players registered

results = []
seasons = ['season-7.2.0']

for season in seasons:
    for spage in range(13):
        results.append(requests.get(f'https://raider.io/api/v1/mythic-plus/runs?season={season}&region=world&dungeon=all&page={spage}').json())


characters = []

for page in range(13):
    for run in range(8):
        for char in range(5):
            characters.append(results[page]['rankings'][run]['run']['roster'][char])

IndexError: list index out of range

In [ ]:
#the solution was to parameterize the values so that it didn't matter if the number of players registered in a run was 5 or not.
num_of_pages = len(results)
characters = []

for page in range(num_of_pages):
    num_of_run = len(results[page]['rankings'])
    for run in range(num_of_run):
        num_of_char = len(results[page]['rankings'][run]['run']['roster'])
        for char in range(num_of_char):
            characters.append(results[page]['rankings'][run]['run']['roster'][char])

## Creating a dataframe for the results

In [ ]:
df = pd.json_normalize(characters)
df

## Dataframe cleaning

In [ ]:
df_drop = df.drop(df.columns.difference(['role', 'season', 'character.class.name', 'character.spec.name', 'character.region.name']), 1)

In [ ]:
df_drop

In [ ]:
df_drop = df_drop.rename(columns = {'character.class.name': 'class', 'character.spec.name': 'spec', 'character.region.name': 'region'})
df_drop

## CSV Export

In [ ]:
df_drop.to_csv('wowzin.csv')